In [91]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

df = pd.read_parquet('./data')
print(f'total dataset: {len(df)} rows')

## filters
# df = df[df['stabilized_spread'] < 10]  # nem tul nagy spread
df = df[df['stabilized_spread'] > 0.5]  # nem tul kis spread
# df = df[abs(df['last_price_delta_since_stabilized']) > 0.1]
# df = df[abs(df['last_price_delta_since_stabilized']) < 8]

# df = df[df['stabilized_amount_mean'] < 0.4]  # nem tul nagy tradek
# df = df[df['stabilized_nr_trades'] < 300]  # nem tul sok trade

df.sort_index(axis=1, inplace=True)

for col in df.columns:
    # if 'nr_trades' in col or 'price_delta' in col and not col.startswith('last'):
    #     df.drop(col, axis=1, inplace=True)
    if 'past' in col:
        df.drop(col, axis=1, inplace=True)
    if '_spread' == col[1:]:
        df.drop(col, axis=1, inplace=True)


y = df['last_price_delta_since_stabilized']
X = df.drop('last_price_delta_since_stabilized', axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.75, test_size=0.25, random_state=0)
X_train

total dataset: 22000 rows


,0_amount_mean,0_nr_trades,0_price_delta,1_amount_mean,1_nr_trades,1_price_delta,2_amount_mean,2_nr_trades,2_price_delta,3_amount_mean,...,3_price_delta,4_amount_mean,4_nr_trades,4_price_delta,stabilized_amount_mean,stabilized_at_ms,stabilized_gasp,stabilized_nr_trades,stabilized_spread,wave_direction
index,,,,,,,,,,,,,,,,,,,,,
0,0.0134,5.0,0.20,0.0161,11.0,0.20,0.0206,13.0,0.20,0.0195,...,0.00,0.0204,15.0,0.00,0.0261,174,-0.0629,17.0,0.53,min
0,0.0016,6.0,0.37,0.0021,8.0,0.37,0.0019,9.0,0.37,0.0041,...,0.00,0.0056,12.0,0.00,0.0061,324,-0.3611,13.0,1.00,max
0,0.0092,5.0,0.70,0.0144,6.0,0.70,0.0128,7.0,0.70,0.0261,...,0.36,0.0371,34.0,0.00,0.0350,347,0.8572,37.0,1.23,min
0,0.0048,16.0,0.10,0.0045,17.0,0.10,0.0044,18.0,0.10,0.0112,...,0.10,0.0121,24.0,0.07,0.0108,790,0.6852,27.0,0.83,min
0,0.0242,5.0,2.17,0.0323,29.0,1.55,0.0424,118.0,0.38,0.0445,...,0.00,0.0440,134.0,0.00,0.0440,314,-0.5969,136.0,2.52,max
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.0039,8.0,0.04,0.0037,9.0,0.04,0.0124,13.0,0.04,0.0110,...,0.00,0.0134,16.0,0.00,0.0127,566,0.3377,17.0,0.54,min
0,0.0460,3.0,0.50,0.0407,4.0,0.50,0.0327,5.0,0.50,0.0236,...,0.50,0.0272,8.0,0.50,0.0192,682,0.8086,15.0,1.26,min
0,0.0680,45.0,1.15,0.0663,48.0,1.15,0.0670,49.0,1.15,0.0678,...,1.13,0.0510,81.0,0.37,0.0447,269,1.1712,101.0,2.43,min


In [92]:
from spreadsurfer.price_engine import FeatureEngineer
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler
# from sklearn.ensemle import RandomForestRegressor

model = CatBoostRegressor(learning_rate=0.01, depth=7, loss_function='MAPE', random_state=0, verbose=False, iterations=1500)
model2 = CatBoostRegressor(learning_rate=0.01, depth=7, loss_function='RMSE', random_state=0, verbose=False, iterations=1500)
trained_model = model

pipeline = Pipeline(steps=[
    ('preprocessor', FeatureEngineer()),
    ('model', model)
])

pipeline2 = Pipeline(steps=[
    ('preprocessor', FeatureEngineer()),
    ('model', model2)
])

pipeline.fit(X_train, y_train)
pipeline2.fit(X_train, y_train)
score = pipeline.score(X_valid, y_valid)
score2 = pipeline2.score(X_valid, y_valid)
display(score)
display(score2)


0.20283706072760477

0.3199839268786232

In [517]:
import catboost
w = catboost.MetricVisualizer('./models', subdirs=True)
w.start()

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

boo


In [37]:
from sklearn.model_selection import GridSearchCV

do_grid_search = False

if do_grid_search:
    X = FeatureEngineer().transform(X)
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

    parameters = {'depth': [6, 7, 8],
                  'learning_rate': [0.015, 0.025],
                  'iterations': [500, 1000, 1500]
                  }
    grid = GridSearchCV(estimator=model, param_grid=parameters, cv=2, n_jobs=-1)
    grid.fit(X_train, y_train)

    print(" Results from Grid Search ")
    print("\n The best estimator across ALL searched params:\n", grid.best_estimator_)
    print("\n The best score across ALL searched params:\n", grid.best_score_)
    print("\n The best parameters across ALL searched params:\n", grid.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:

 The best score across ALL searched params:
 0.41200864218441446

 The best parameters across ALL searched params:
 {'depth': 7, 'iterations': 1500, 'learning_rate': 0.025}


In [98]:
from spreadsurfer import now_isoformat

save = True
if save:
    model2.save_model(f'./models/{now_isoformat()}-score-{1000 * round(score, 3)}.cat')


In [94]:
input_from_log = """


2023-01-31 22:20:30.837 | data     | spreadsurfer.datacollect:start:97 - wave 22343 collected: {'0_amount_mean': 0.004, '0_nr_trades': 8.0, '0_price_delta': 0.44, '0_spread': 0.84, '1_amount_mean': 0.0089, '1_nr_trades': 12.0, '1_price_delta': 0.44, '1_spread': 1.12, '2_amount_mean': 0.0108, '2_nr_trades': 14.0, '2_price_delta': 0.06, '2_spread': 1.5, '3_amount_mean': 0.0141, '3_nr_trades': 16.0, '3_price_delta': 0.0, '3_spread': 1.56, '4_amount_mean': 0.0133, '4_nr_trades': 17.0, '4_price_delta': 0.0, '4_spread': 1.56, 'last_price_delta_since_stabilized': -0.42, 'past_final_price_0': -9.470000000001164, 'past_final_price_1': -9.279999999998836, 'past_final_price_2': -8.970000000001164, 'past_final_price_3': -9.369999999998981, 'past_final_price_4': -7.819999999999709, 'past_final_price_5': -8.5, 'past_final_price_6': -6.669999999998254, 'past_final_price_7': -6.430000000000291, 'past_final_price_8': -2.970000000001164, 'past_final_price_9': 0.0, 'stabilized_amount_mean': 0.0126, 'stabilized_at_ms': 280, 'stabilized_gasp': -0.6397, 'stabilized_nr_trades': 18.0, 'stabilized_spread': 1.56, 'wave_direction': None}
2023-01-31 22:20:31.282 | data     | spreadsurfer.datacollect:start:97 - wave 22344 collected: {'0_amount_mean': 0.016, '0_nr_trades': 54.0, '0_price_delta': 0.13, '0_spread': 1.98, '1_amount_mean': 0.0155, '1_nr_trades': 56.0, '1_price_delta': 0.13, '1_spread': 1.98, '2_amount_mean': 0.0174, '2_nr_trades': 58.0, '2_price_delta': 0.13, '2_spread': 1.98, '3_amount_mean': 0.017, '3_nr_trades': 65.0, '3_price_delta': 0.13, '3_spread': 1.98, '4_amount_mean': 0.0184, '4_nr_trades': 75.0, '4_price_delta': 0.0, '4_spread': 2.11, 'last_price_delta_since_stabilized': 0.0, 'past_final_price_0': -7.629999999997381, 'past_final_price_1': -7.319999999999709, 'past_final_price_2': -7.719999999997526, 'past_final_price_3': -6.169999999998254, 'past_final_price_4': -6.849999999998545, 'past_final_price_5': -5.019999999996799, 'past_final_price_6': -4.779999999998836, 'past_final_price_7': -1.319999999999709, 'past_final_price_8': 1.6500000000014552, 'past_final_price_9': 0.0, 'stabilized_amount_mean': 0.0179, 'stabilized_at_ms': 675, 'stabilized_gasp': 1.1126, 'stabilized_nr_trades': 78.0, 'stabilized_spread': 2.11, 'wave_direction': 'min'}
2023-01-31 22:20:32.027 | data     | spreadsurfer.datacollect:start:97 - wave 22345 collected: {'0_amount_mean': 0.0014, '0_nr_trades': 6.0, '0_price_delta': 0.25, '0_spread': 0.39, '1_amount_mean': 0.0109, '1_nr_trades': 7.0, '1_price_delta': 0.25, '1_spread': 0.39, '2_amount_mean': 0.0096, '2_nr_trades': 8.0, '2_price_delta': 0.2, '2_spread': 0.44, '3_amount_mean': 0.0086, '3_nr_trades': 9.0, '3_price_delta': 0.0, '3_spread': 0.64, '4_amount_mean': 0.0079, '4_nr_trades': 10.0, '4_price_delta': 0.0, '4_spread': 0.64, 'last_price_delta_since_stabilized': 1.16, 'past_final_price_0': -6.389999999999418, 'past_final_price_1': -6.789999999997235, 'past_final_price_2': -5.239999999997963, 'past_final_price_3': -5.919999999998254, 'past_final_price_4': -4.0899999999965075, 'past_final_price_5': -3.849999999998545, 'past_final_price_6': -0.3899999999994179, 'past_final_price_7': 2.5800000000017462, 'past_final_price_8': 0.930000000000291, 'past_final_price_9': 0.0, 'stabilized_amount_mean': 0.0073, 'stabilized_at_ms': 101, 'stabilized_gasp': 0.1353, 'stabilized_nr_trades': 11.0, 'stabilized_spread': 0.64, 'wave_direction': 'min'}
2023-01-31 22:20:32.974 | data     | spreadsurfer.datacollect:start:97 - wave 22346 collected: {'0_amount_mean': 0.067, '0_nr_trades': 4.0, '0_price_delta': 0.11, '0_spread': 0.42, '1_amount_mean': 0.0538, '1_nr_trades': 5.0, '1_price_delta': 0.0, '1_spread': 0.53, '2_amount_mean': 0.0449, '2_nr_trades': 6.0, '2_price_delta': 0.0, '2_spread': 0.53, '3_amount_mean': 0.0342, '3_nr_trades': 8.0, '3_price_delta': 0.0, '3_spread': 0.53, '4_amount_mean': 0.0312, '4_nr_trades': 10.0, '4_price_delta': 0.0, '4_spread': 0.58, 'last_price_delta_since_stabilized': -0.1, 'past_final_price_0': -6.389999999999418, 'past_final_price_1': -6.789999999997235, 'past_final_price_2': -5.239999999997963, 'past_final_price_3': -5.919999999998254, 'past_final_price_4': -4.0899999999965075, 'past_final_price_5': -3.849999999998545, 'past_final_price_6': -0.3899999999994179, 'past_final_price_7': 2.5800000000017462, 'past_final_price_8': 0.930000000000291, 'past_final_price_9': 0.0, 'stabilized_amount_mean': 0.026, 'stabilized_at_ms': 626, 'stabilized_gasp': -0.5699, 'stabilized_nr_trades': 13.0, 'stabilized_spread': 0.62, 'wave_direction': None}
2023-01-31 22:20:33.095 | data     | spreadsurfer.datacollect:start:97 - wave 22347 collected: {'0_amount_mean': 0.0312, '0_nr_trades': 10.0, '0_price_delta': 0.1, '0_spread': 0.58, '1_amount_mean': 0.026, '1_nr_trades': 13.0, '1_price_delta': 0.1, '1_spread': 0.62, '2_amount_mean': 0.0242, '2_nr_trades': 14.0, '2_price_delta': 0.1, '2_spread': 0.62, '3_amount_mean': 0.0226, '3_nr_trades': 15.0, '3_price_delta': 0.1, '3_spread': 0.62, '4_amount_mean': 0.0223, '4_nr_trades': 17.0, '4_price_delta': 0.0, '4_spread': 0.72, 'last_price_delta_since_stabilized': 0.0, 'past_final_price_0': -7.849999999998545, 'past_final_price_1': -6.299999999999272, 'past_final_price_2': -6.979999999999563, 'past_final_price_3': -5.149999999997817, 'past_final_price_4': -4.9099999999998545, 'past_final_price_5': -1.4500000000007276, 'past_final_price_6': 1.5200000000004366, 'past_final_price_7': -0.13000000000101863, 'past_final_price_8': -1.0600000000013097, 'past_final_price_9': 0.0, 'stabilized_amount_mean': 0.0249, 'stabilized_at_ms': 792, 'stabilized_gasp': -0.2825, 'stabilized_nr_trades': 18.0, 'stabilized_spread': 0.72, 'wave_direction': 'max'}
2023-01-31 22:20:33.701 | data     | spreadsurfer.datacollect:start:97 - wave 22348 collected: {'0_amount_mean': 0.003, '0_nr_trades': 3.0, '0_price_delta': 0.43, '0_spread': 0.19, '1_amount_mean': 0.0023, '1_nr_trades': 4.0, '1_price_delta': 0.43, '1_spread': 0.19, '2_amount_mean': 0.0043, '2_nr_trades': 5.0, '2_price_delta': 0.01, '2_spread': 0.61, '3_amount_mean': 0.0044, '3_nr_trades': 6.0, '3_price_delta': 0.0, '3_spread': 0.62, '4_amount_mean': 0.0045, '4_nr_trades': 7.0, '4_price_delta': 0.0, '4_spread': 0.62, 'last_price_delta_since_stabilized': 0.0, 'past_final_price_0': -7.849999999998545, 'past_final_price_1': -6.299999999999272, 'past_final_price_2': -6.979999999999563, 'past_final_price_3': -5.149999999997817, 'past_final_price_4': -4.9099999999998545, 'past_final_price_5': -1.4500000000007276, 'past_final_price_6': 1.5200000000004366, 'past_final_price_7': -0.13000000000101863, 'past_final_price_8': -1.0600000000013097, 'past_final_price_9': 0.0, 'stabilized_amount_mean': 0.0042, 'stabilized_at_ms': 153, 'stabilized_gasp': -0.3461, 'stabilized_nr_trades': 8.0, 'stabilized_spread': 0.62, 'wave_direction': None}
2023-01-31 22:20:34.001 | data     | spreadsurfer.datacollect:start:97 - wave 22349 collected: {'0_amount_mean': 0.0044, '0_nr_trades': 6.0, '0_price_delta': 0.0, '0_spread': 0.62, '1_amount_mean': 0.0045, '1_nr_trades': 7.0, '1_price_delta': 0.0, '1_spread': 0.62, '2_amount_mean': 0.0042, '2_nr_trades': 8.0, '2_price_delta': 0.0, '2_spread': 0.62, '3_amount_mean': 0.004, '3_nr_trades': 9.0, '3_price_delta': 0.0, '3_spread': 0.63, '4_amount_mean': 0.0036, '4_nr_trades': 10.0, '4_price_delta': 0.0, '4_spread': 0.65, 'last_price_delta_since_stabilized': -0.18, 'past_final_price_0': -7.849999999998545, 'past_final_price_1': -6.299999999999272, 'past_final_price_2': -6.979999999999563, 'past_final_price_3': -5.149999999997817, 'past_final_price_4': -4.9099999999998545, 'past_final_price_5': -1.4500000000007276, 'past_final_price_6': 1.5200000000004366, 'past_final_price_7': -0.13000000000101863, 'past_final_price_8': -1.0600000000013097, 'past_final_price_9': 0.0, 'stabilized_amount_mean': 0.0031, 'stabilized_at_ms': 550, 'stabilized_gasp': -0.4828, 'stabilized_nr_trades': 12.0, 'stabilized_spread': 0.82, 'wave_direction': None}
2023-01-31 22:20:35.825 | data     | spreadsurfer.datacollect:start:97 - wave 22350 collected: {'0_amount_mean': 0.0013, '0_nr_trades': 3.0, '0_price_delta': 0.81, '0_spread': 0.0, '1_amount_mean': 0.0011, '1_nr_trades': 4.0, '1_price_delta': 0.41, '1_spread': 0.4, '2_amount_mean': 0.0089, '2_nr_trades': 13.0, '2_price_delta': 0.03, '2_spread': 0.78, '3_amount_mean': 0.017, '3_nr_trades': 15.0, '3_price_delta': 0.01, '3_spread': 0.8, '4_amount_mean': 0.0157, '4_nr_trades': 17.0, '4_price_delta': 0.0, '4_spread': 0.81, 'last_price_delta_since_stabilized': -0.08, 'past_final_price_0': -7.680000000000291, 'past_final_price_1': -5.849999999998545, 'past_final_price_2': -5.610000000000582, 'past_final_price_3': -2.150000000001455, 'past_final_price_4': 0.819999999999709, 'past_final_price_5': -0.8300000000017462, 'past_final_price_6': -1.7600000000020373, 'past_final_price_7': -0.7000000000007276, 'past_final_price_8': 0.029999999998835847, 'past_final_price_9': 0.0, 'stabilized_amount_mean': 0.0157, 'stabilized_at_ms': 266, 'stabilized_gasp': -0.1344, 'stabilized_nr_trades': 18.0, 'stabilized_spread': 0.81, 'wave_direction': None}
2023-01-31 22:20:36.045 | data     | spreadsurfer.datacollect:start:97 - wave 22351 collected: {'0_amount_mean': 0.0157, '0_nr_trades': 18.0, '0_price_delta': 0.29, '0_spread': 0.81, '1_amount_mean': 0.0149, '1_nr_trades': 19.0, '1_price_delta': 0.29, '1_spread': 0.89, '2_amount_mean': 0.0143, '2_nr_trades': 20.0, '2_price_delta': 0.29, '2_spread': 0.89, '3_amount_mean': 0.0136, '3_nr_trades': 21.0, '3_price_delta': 0.29, '3_spread': 0.89, '4_amount_mean': 0.0178, '4_nr_trades': 27.0, '4_price_delta': 0.27, '4_spread': 0.91, 'last_price_delta_since_stabilized': 0.24, 'past_final_price_0': -4.969999999997526, 'past_final_price_1': -4.729999999999563, 'past_final_price_2': -1.2700000000004366, 'past_final_price_3': 1.7000000000007276, 'past_final_price_4': 0.049999999999272404, 'past_final_price_5': -0.8800000000010186, 'past_final_price_6': 0.18000000000029104, 'past_final_price_7': 0.9099999999998545, 'past_final_price_8': 0.8800000000010186, 'past_final_price_9': 0.0, 'stabilized_amount_mean': 0.0168, 'stabilized_at_ms': 621, 'stabilized_gasp': 0.9153, 'stabilized_nr_trades': 29.0, 'stabilized_spread': 1.18, 'wave_direction': 'min'}
2023-01-31 22:20:37.032 | data     | spreadsurfer.datacollect:start:97 - wave 22352 collected: {'0_amount_mean': 0.0018, '0_nr_trades': 3.0, '0_price_delta': 0.55, '0_spread': 0.41, '1_amount_mean': 0.0015, '1_nr_trades': 4.0, '1_price_delta': 0.55, '1_spread': 0.41, '2_amount_mean': 0.0013, '2_nr_trades': 5.0, '2_price_delta': 0.55, '2_spread': 0.41, '3_amount_mean': 0.0012, '3_nr_trades': 6.0, '3_price_delta': 0.55, '3_spread': 0.41, '4_amount_mean': 0.0011, '4_nr_trades': 7.0, '4_price_delta': 0.55, '4_spread': 0.41, 'last_price_delta_since_stabilized': 0.0, 'past_final_price_0': -5.360000000000582, 'past_final_price_1': -1.9000000000014552, 'past_final_price_2': 1.069999999999709, 'past_final_price_3': -0.5800000000017462, 'past_final_price_4': -1.5100000000020373, 'past_final_price_5': -0.4500000000007276, 'past_final_price_6': 0.27999999999883585, 'past_final_price_7': 0.25, 'past_final_price_8': -0.6300000000010186, 'past_final_price_9': 0.0, 'stabilized_amount_mean': 0.0011, 'stabilized_at_ms': 149, 'stabilized_gasp': 0.0871, 'stabilized_nr_trades': 10.0, 'stabilized_spread': 0.96, 'wave_direction': 'max'}
2023-01-31 22:20:37.497 | data     | spreadsurfer.datacollect:start:97 - wave 22353 collected: {'0_amount_mean': 0.0035, '0_nr_trades': 4.0, '0_price_delta': 0.0, '0_spread': 0.08, '1_amount_mean': 0.0029, '1_nr_trades': 5.0, '1_price_delta': 0.0, '1_spread': 0.19, '2_amount_mean': 0.0024, '2_nr_trades': 6.0, '2_price_delta': 0.0, '2_spread': 0.19, '3_amount_mean': 0.0022, '3_nr_trades': 8.0, '3_price_delta': 0.0, '3_spread': 0.33, '4_amount_mean': 0.0024, '4_nr_trades': 9.0, '4_price_delta': 0.0, '4_spread': 0.33, 'last_price_delta_since_stabilized': -0.19, 'past_final_price_0': -5.360000000000582, 'past_final_price_1': -1.9000000000014552, 'past_final_price_2': 1.069999999999709, 'past_final_price_3': -0.5800000000017462, 'past_final_price_4': -1.5100000000020373, 'past_final_price_5': -0.4500000000007276, 'past_final_price_6': 0.27999999999883585, 'past_final_price_7': 0.25, 'past_final_price_8': -0.6300000000010186, 'past_final_price_9': 0.0, 'stabilized_amount_mean': 0.0045, 'stabilized_at_ms': 148, 'stabilized_gasp': 0.2558, 'stabilized_nr_trades': 10.0, 'stabilized_spread': 0.33, 'wave_direction': None}
2023-01-31 22:20:37.783 | data     | spreadsurfer.datacollect:start:97 - wave 22354 collected: {'0_amount_mean': 0.0024, '0_nr_trades': 6.0, '0_price_delta': 0.19, '0_spread': 0.19, '1_amount_mean': 0.0022, '1_nr_trades': 8.0, '1_price_delta': 0.19, '1_spread': 0.33, '2_amount_mean': 0.0024, '2_nr_trades': 9.0, '2_price_delta': 0.19, '2_spread': 0.33, '3_amount_mean': 0.0045, '3_nr_trades': 10.0, '3_price_delta': 0.19, '3_spread': 0.33, '4_amount_mean': 0.0043, '4_nr_trades': 11.0, '4_price_delta': 0.0, '4_spread': 0.52, 'last_price_delta_since_stabilized': 0.0, 'past_final_price_0': -5.360000000000582, 'past_final_price_1': -1.9000000000014552, 'past_final_price_2': 1.069999999999709, 'past_final_price_3': -0.5800000000017462, 'past_final_price_4': -1.5100000000020373, 'past_final_price_5': -0.4500000000007276, 'past_final_price_6': 0.27999999999883585, 'past_final_price_7': 0.25, 'past_final_price_8': -0.6300000000010186, 'past_final_price_9': 0.0, 'stabilized_amount_mean': 0.0046, 'stabilized_at_ms': 502, 'stabilized_gasp': -0.0125, 'stabilized_nr_trades': 12.0, 'stabilized_spread': 0.52, 'wave_direction': None}
2023-01-31 22:20:38.931 | data     | spreadsurfer.datacollect:start:97 - wave 22355 collected: {'0_amount_mean': 0.0803, '0_nr_trades': 5.0, '0_price_delta': 0.0, '0_spread': 0.07, '1_amount_mean': 0.0575, '1_nr_trades': 7.0, '1_price_delta': 0.0, '1_spread': 0.44, '2_amount_mean': 0.0449, '2_nr_trades': 9.0, '2_price_delta': 0.0, '2_spread': 0.47, '3_amount_mean': 0.037, '3_nr_trades': 11.0, '3_price_delta': 0.0, '3_spread': 0.68, '4_amount_mean': 0.0341, '4_nr_trades': 12.0, '4_price_delta': 0.0, '4_spread': 0.68, 'last_price_delta_since_stabilized': -0.08, 'past_final_price_0': -2.1300000000010186, 'past_final_price_1': 0.8400000000001455, 'past_final_price_2': -0.8100000000013097, 'past_final_price_3': -1.7400000000016007, 'past_final_price_4': -0.680000000000291, 'past_final_price_5': 0.049999999999272404, 'past_final_price_6': 0.020000000000436557, 'past_final_price_7': -0.8600000000005821, 'past_final_price_8': -0.22999999999956344, 'past_final_price_9': 0.0, 'stabilized_amount_mean': 0.0363, 'stabilized_at_ms': 271, 'stabilized_gasp': -0.1038, 'stabilized_nr_trades': 13.0, 'stabilized_spread': 0.68, 'wave_direction': None}
2023-01-31 22:20:39.276 | data     | spreadsurfer.datacollect:start:97 - wave 22356 collected: {'0_amount_mean': 0.037, '0_nr_trades': 11.0, '0_price_delta': 0.18, '0_spread': 0.68, '1_amount_mean': 0.0341, '1_nr_trades': 12.0, '1_price_delta': 0.18, '1_spread': 0.68, '2_amount_mean': 0.0363, '2_nr_trades': 13.0, '2_price_delta': 0.18, '2_spread': 0.68, '3_amount_mean': 0.0339, '3_nr_trades': 14.0, '3_price_delta': 0.18, '3_spread': 0.68, '4_amount_mean': 0.0297, '4_nr_trades': 16.0, '4_price_delta': 0.1, '4_spread': 0.76, 'last_price_delta_since_stabilized': 0.0, 'past_final_price_0': 1.0, 'past_final_price_1': -0.6500000000014552, 'past_final_price_2': -1.5800000000017462, 'past_final_price_3': -0.5200000000004366, 'past_final_price_4': 0.20999999999912689, 'past_final_price_5': 0.18000000000029104, 'past_final_price_6': -0.7000000000007276, 'past_final_price_7': -0.06999999999970896, 'past_final_price_8': 0.15999999999985448, 'past_final_price_9': 0.0, 'stabilized_amount_mean': 0.0271, 'stabilized_at_ms': 377, 'stabilized_gasp': -0.4314, 'stabilized_nr_trades': 18.0, 'stabilized_spread': 0.86, 'wave_direction': 'max'}
2023-01-31 22:20:41.016 | data     | spreadsurfer.datacollect:start:97 - wave 22357 collected: {'0_amount_mean': 0.1542, '0_nr_trades': 107.0, '0_price_delta': 0.12, '0_spread': 3.0, '1_amount_mean': 0.1544, '1_nr_trades': 109.0, '1_price_delta': 0.12, '1_spread': 3.0, '2_amount_mean': 0.1518, '2_nr_trades': 111.0, '2_price_delta': 0.0, '2_spread': 3.12, '3_amount_mean': 0.1504, '3_nr_trades': 112.0, '3_price_delta': 0.0, '3_spread': 3.12, '4_amount_mean': 0.1491, '4_nr_trades': 113.0, '4_price_delta': 0.0, '4_spread': 3.12, 'last_price_delta_since_stabilized': 0.75, 'past_final_price_0': 3.1199999999989814, 'past_final_price_1': 4.180000000000291, 'past_final_price_2': 4.9099999999998545, 'past_final_price_3': 4.880000000001019, 'past_final_price_4': 4.0, 'past_final_price_5': 4.630000000001019, 'past_final_price_6': 4.860000000000582, 'past_final_price_7': 4.700000000000728, 'past_final_price_8': 4.049999999999272, 'past_final_price_9': 0.0, 'stabilized_amount_mean': 0.1478, 'stabilized_at_ms': 286, 'stabilized_gasp': 0.1456, 'stabilized_nr_trades': 114.0, 'stabilized_spread': 3.12, 'wave_direction': 'min'}


"""
from timeit import timeit

def predict_samples():
    mae = 0
    mae2 = 0
    count = 0
    for line in input_from_log.split('\n'):
        if not line: continue
        count += 1
        json = '{' + line.split('{')[1]
        sample = pd.DataFrame([eval(json)])

        real = sample.last_price_delta_since_stabilized[0]
        print('expected: ', real)
        sample.drop('last_price_delta_since_stabilized', axis=1, inplace=True)

        timeit(lambda: pipeline.predict(sample), number=1) * 1000
        guess = pipeline.predict(sample)[0]
        guess2 = pipeline2.predict(sample)[0]
        print('predict: ', guess)
        print('predict2: ', guess2)
        mae += abs(real - guess)
        mae2 += abs(real - guess2)
        print()
    display(f'MAE: {mae / count}')
    display(f'MAE2: {mae2 / count}')
predict_samples()

expected:  -0.42
predict:  -0.004527232013073203
predict2:  -0.11508573131767105

expected:  0.0
predict:  0.017037201072637335
predict2:  0.24090907536948214

expected:  1.16
predict:  0.400072277001099
predict2:  0.9001358393516005

expected:  -0.1
predict:  -0.11689968358976079
predict2:  -0.18311505087602828

expected:  0.0
predict:  -0.1492188860785844
predict2:  -0.46414033515425945

expected:  0.0
predict:  -0.008060582732293245
predict2:  -0.13631446342987458

expected:  -0.18
predict:  -0.13557536747458984
predict2:  -0.20378007915927973

expected:  -0.08
predict:  -0.004628002971178673
predict2:  -0.11134086905746869

expected:  0.24
predict:  0.1450635879986768
predict2:  0.33250428432370693

expected:  0.0
predict:  -0.1492188860785844
predict2:  -0.46414033515425945

expected:  -0.19
predict:  -0.16847510043955144
predict2:  -0.30930224397604844

expected:  0.0
predict:  -0.012970525751434899
predict2:  -0.08513683626927235

expected:  -0.08
predict:  -0.14854506993539027


'MAE: 0.17616474077788627'

'MAE2: 0.21304120460492412'

In [100]:
sample = pd.DataFrame([
    {'0_amount_mean': 0.007, '0_nr_trades': 12.0, '0_price_delta': 0.91, '0_spread': 1.15, '1_amount_mean': 0.007, '1_nr_trades': 13.0, '1_price_delta': 0.91, '1_spread': 1.52, '2_amount_mean': 0.006, '2_nr_trades': 14.0,
     '2_price_delta': 0.91, '2_spread': 1.52, '3_amount_mean': 0.006, '3_nr_trades': 16.0, '3_price_delta': 0.69, '3_spread': 1.74, '4_amount_mean': 0.005, '4_nr_trades': 17.0, '4_price_delta': 0.69, '4_spread': 1.74,
     'last_price_delta_since_stabilized': -0.87, 'past_final_price_0': -0.11999999999898137, 'past_final_price_1': 0.25, 'past_final_price_2': -0.020000000000436557, 'past_final_price_3': -1.7400000000016007,
     'past_final_price_4': 0.06999999999970896, 'past_final_price_5': -2.7299999999995634, 'past_final_price_6': -3.9099999999998545, 'past_final_price_7': 3.360000000000582, 'past_final_price_8': -0.06000000000130967,
     'past_final_price_9': 0.0, 'stabilized_amount_mean': 0.007, 'stabilized_at_ms': 369, 'stabilized_nr_trades': 18.0, 'stabilized_spread': 2.43, 'wave_direction': 'max'}
])

print('expected: ', sample.last_price_delta_since_stabilized.real())
sample.drop('last_price_delta_since_stabilized', axis=1, inplace=True)

from timeit import timeit

timeit(lambda: pipeline.predict(sample), number=1) * 1000
print('predict: ', pipeline.predict(sample)[0])


AttributeError: 'Series' object has no attribute 'real'

In [97]:
from glob import glob

cat_filename = glob('*.cat')[0]
cat_filename

model = CatBoostRegressor()
model.load_model(fname=cat_filename)
print(f'catboost model loaded from {cat_filename}')


pipeline = Pipeline(steps=[
    ('preprocessor', FeatureEngineer()),
    ('model', model)
])

predict_samples()


catboost model loaded from 2023-01-31T22:22:38-score-203.0.cat
expected:  -0.42
predict:  -0.004527232013073203
predict2:  -0.11508573131767105

expected:  0.0
predict:  0.017037201072637335
predict2:  0.24090907536948214

expected:  1.16
predict:  0.400072277001099
predict2:  0.9001358393516005

expected:  -0.1
predict:  -0.11689968358976079
predict2:  -0.18311505087602828

expected:  0.0
predict:  -0.1492188860785844
predict2:  -0.46414033515425945

expected:  0.0
predict:  -0.008060582732293245
predict2:  -0.13631446342987458

expected:  -0.18
predict:  -0.13557536747458984
predict2:  -0.20378007915927973

expected:  -0.08
predict:  -0.004628002971178673
predict2:  -0.11134086905746869

expected:  0.24
predict:  0.1450635879986768
predict2:  0.33250428432370693

expected:  0.0
predict:  -0.1492188860785844
predict2:  -0.46414033515425945

expected:  -0.19
predict:  -0.16847510043955144
predict2:  -0.30930224397604844

expected:  0.0
predict:  -0.012970525751434899
predict2:  -0.0851

'MAE: 0.17616474077788627'

'MAE2: 0.21304120460492412'